# Analyse deposit event based profit calculation issue in

See https://github.com/tradingstrategy-ai/trade-executor/issues/667


In [2]:
from tradeexecutor.utils.state_downloader import download_state
import os

url = "https://polygon-multipair-momentum.tradingstrategy.ai"
name = "polygon-multipair-momentum.json"

if not os.path.exists(f"/tmp/{name}"):
    state = download_state(url)
    state.write_json_file(f"/tmp/{name}")


Downloaded state for https://polygon-multipair-momentum.tradingstrategy.ai, total 21,511,533 chars


In [3]:
from tradeexecutor.state.state import State

state = State.read_json_file(f"/tmp/{name}")


## Balance checks

Check for current balances and abnormal balance update events.

In [4]:
portfolio = state.portfolio
asset, exchange_rate = state.portfolio.get_default_reserve_asset()
position = portfolio.get_default_reserve_position()
print(f"Number of reserve assets: {len(portfolio.reserves)}")
print(f"Reserves: {asset} at {exchange_rate} {asset.token_symbol} / USD")
print(f"Amount: {position.get_quantity()} {asset.token_symbol}")
print(f"Equity: {position.get_total_equity()} USD")
print(f"Value: {position.get_value()} USD")
print(f"Balance updates: {position.get_base_token_balance_update_quantity()} {asset.token_symbol}")
print(f"Number of balance updates events: {len(position.balance_updates)}")
print(f"Number of events references in the portfolio overall: {len(state.sync.treasury.balance_update_refs)}")

Number of reserve assets: 1
Reserves: <USDC at 0x2791bca1f2de4661ed88a30c99a7a9449aa84174> at 1.0 USDC / USD
Amount: 71.875852 USDC
Equity: 71.875852 USD
Value: 71.875852 USD
Balance updates: 895131.464809 USDC
Number of balance updates events: 12193
Number of events references in the portfolio overall: 0


# Vault sync status

In [5]:
treasury = state.sync.treasury
deployment = state.sync.deployment

print(f"Vault deployment: {deployment}")
print(f"Treasury sync status: {treasury}")

Vault deployment: <Deployment chain:polygon address:0x8C95f9BB3bD2971Dd7D8F25D4F5D98e937A82F36 name:None token:None>
Treasury sync status: <Treasury updated:2023-12-13 18:57:01 cycle:2023-12-13 18:57:00 block scanned:51,093,617 refs:0>


## Individual events

For the last analysed executor.

In [6]:
from tradeexecutor.analysis.position import display_reserve_position_events

df = display_reserve_position_events(position)
display(df)


,Cause,At,Quantity,Dollar value,Address,Notes
6084,deposit,2023-11-07 13:53:49,61.19656,61.196560,,
6085,deposit,2023-11-07 13:54:00,61.19656,61.196560,,
6086,deposit,2023-11-07 13:57:00,61.19656,61.196560,,
6087,deposit,2023-11-07 14:20:32,61.19656,61.196560,,
6088,deposit,2023-11-07 14:21:00,61.19656,61.196560,,
...,...,...,...,...,...,...
18272,deposit,2023-12-13 18:45:00,71.875852,71.875852,,
18273,deposit,2023-12-13 18:48:00,71.875852,71.875852,,
18274,deposit,2023-12-13 18:51:00,71.875852,71.875852,,
18275,deposit,2023-12-13 18:54:00,71.875852,71.875852,,


## Position profit history

In [7]:
from io import StringIO
from typing import Iterable, List
import pandas as pd

from tradeexecutor.state.position import TradingPosition
from tradeexecutor.state import position

positions = state.portfolio.closed_positions.values()

data = [(p.closed_at, p.position_id, p.get_realised_profit_percent() * 100, p.is_repaired()) for p in positions]

df = pd.DataFrame(data, columns=("date", "position id", "position profit %", "failed trades"))
df = df.set_index("date")

df = df.sort_values("position profit %", ascending=True)

pd.set_option('display.max_rows', 100)

df


,position id,position profit %,failed trades
date,,,
2023-10-24 22:56:40,3,-55.051863,False
2023-11-21 18:10:49,88,-9.023649,False
2023-12-10 15:08:10,105,-6.000636,False
2023-10-26 15:04:51,11,-5.522145,False
2023-11-17 12:03:59,81,-5.166125,False
...,...,...,...
2023-11-14 05:27:13,74,3.482485,False
2023-11-02 01:30:16,33,5.287333,False
2023-10-28 10:36:22,16,5.961395,False


## Check prolematic position

In [8]:
p: TradingPosition
p = state.portfolio.closed_positions[22]
print("Position", p)
print("Realised profit", p.get_realised_profit_percent())
print("Avg buy", p.get_average_buy())
print("Avg sell", p.get_average_sell())
print("Buy value", p.get_buy_value())
print("Sell value", p.get_sell_value())
print("Balance updates")
for id, bu in p.balance_updates.items():
    print(bu)

Position <Closed position #22 <Pair SAND-WMATIC spot_market_hold at 0x369582d2010b6ed950b571f4101e3bb9b554876f (0.3000% fee) on exchange 0x5757371414417b8c6caad45baef941abc7d3ab32> $4.897948>
Realised profit 0.010607911721398411
Avg buy 0.35133529775086475
Avg sell 0.3550622315740171
Buy value 4.897948
Sell value 4.949905
Balance updates
